In [1]:
import sys
import os 

os.environ["CONFIG_PATHS"] = "../../configs/small_model.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../../data/moves_10"'
sys.path.append("../../src")

import matplotlib.pyplot as plt
import scipy.stats

from analysis import load_events, filter_events

In [2]:
events = load_events("../../data/2024-11-20_14-10-24-survival/logs_2024-11-20_14-37-39_995738.txt")

Start time: 1732141301.009999


In [4]:
game_result_events = filter_events(events, "game_result")
game_results = [event.params for event in game_result_events]

players = set()
for game_result in game_results:
    for player, _ in game_result:
        players.add(player)

print(len(game_results))

689


In [5]:
elos = {player: 1000 for player in players}

elo_K = 32.0
elo_base = 400.0

for game_result in game_results:
    game_players = [player_name for player_name, _ in game_result]
    Q_player = [10 ** (elos[player] / elo_base) for player in game_players]
    Q_total = sum(Q_player)
    E_player = [Q / Q_total for Q in Q_player]

    for i, (player, result) in enumerate(game_result):
        elos[player] += elo_K * (result - E_player[i])

    print(elos)

In [6]:
elos

{'s313486': 1078.2858299207712,
 's0': 595.6623282496626,
 's208119': 1162.935486330412,
 's105769': 1163.1163554991563}

In [7]:
for player, E in zip(game_players, E_player):
    print(player, "\t", E)

s0 	 0.014071577012895747
s313486 	 0.19469341975463678
s208119 	 0.39539536118851615
s105769 	 0.3958396420439512


In [9]:
results = {}
for game_result in game_results:
    for player, result in game_result:
        results[player] = results.get(player, 0) + result

total_results = sum(results.values())
for player in results:
    results[player] = results[player] / total_results

results

{'s208119': 0.33272859216255446,
 's313486': 0.2674165457184325,
 's0': 0.009796806966618287,
 's105769': 0.3900580551523948}